In [115]:
import boto3
import time
import io
import pandas as pd
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import numpy as np
import gmaps
import gmaps.datasets
import requests
import censusgeocode

s3 = boto3.client('s3')

knoxdata_bucket_name = 'knox-data-temp-bucket'
knox_census_bucket = 'knox-data-census'
pilot_filename = 'PILOT_summary.xlsx'
facad_filename = 'facade_grant_recipients.xls'
GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

In [255]:
def read_xl_s3(bucket_name, file_name):
    obj = s3.get_object(Bucket=bucket_name, Key=file_name)
    data = obj['Body'].read()
    return pd.read_excel(io.BytesIO(data), encoding='utf-8')

def get_gpass(filenam, password_type):
    with open(filenam) as f:
        cfile = f.readlines()
    creds = [x.strip().split(':') for x in cfile]
    for i in creds:
        if i[0] == password_type:
            password = i[1]
    return password

def read_csv_s3(bucket_name, file_name):
    obj = s3.get_object(Bucket=bucket_name, Key=file_name)
    data = obj['Body'].read()
    return pd.read_csv(io.BytesIO(data), encoding='utf-8')

def get_latlong_knox(df, address_col):
    new_df = []
    for rownum, row in df.iterrows():
        params = {
            'address': row[address_col]+', knoxville, tn',
            'region': 'us',
            'key':gpass
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Set null vars for writing
        zip_code = 'NA'
        long_address = 'NA'
        # Validate results
        if len(res['results']) != 0:
            result = res['results'][0]
            for i in result['address_components']:
                for t in i['types']:
                    if t == 'postal_code':
                        zip_code = int(i['short_name'])
            if 'formatted_address' in result:
                long_address = result['formatted_address']
            new_df.append([rownum, zip_code, long_address,
                         result['geometry']['location']['lat'], result['geometry']['location']['lng']])
        else:
            new_df.append([rownum, None, None, None, None])
            continue
    
    return df.merge(pd.DataFrame(new_df, columns = ['index', 'zip_code', 'address_formatted', 
                                             'lat', 'lon']).drop(columns=['index']),
             how = 'left', left_index=True, right_index=True)

In [117]:
gpass = get_gpass('/home/ubuntu/.google/googleapi.txt', 'geolocationapi')
gmaps.configure(api_key=gpass)

In [118]:
%%time
# Import PILOT data, format, and add lat/long from google api
pilot_df = read_xl_s3(knoxdata_bucket_name, pilot_filename)
pilot_df = pilot_df[~pilot_df.isnull().any(axis=1)]
pilot_df.columns = [i.lower().strip() for i in pilot_df.columns]
pilot_df = get_latlong_knox(pilot_df, 'address')
pilot_df['source'] = 'pilot'

CPU times: user 517 ms, sys: 25.4 ms, total: 543 ms
Wall time: 15.1 s


In [119]:
%%time
# Import facad data, format, and add lat/long from google api
facad_df = read_xl_s3(knoxdata_bucket_name, facad_filename)
facad_df = facad_df[~facad_df.isnull().ADDRESS]
facad_df.columns = [i.lower() for i in facad_df.columns]
facad_df['status'] = facad_df['status'].str.strip(')').str.replace('(','')
facad_df['complete_date'] = facad_df['status'].str.extract('(\d+.*)')
facad_df['stat'] = facad_df['status'].str.extract('(\D+)')
facad_df['address_new'] = facad_df['address'].str.replace(r"\s\(.*\)","")
facad_df = get_latlong_knox(facad_df, 'address_new')
facad_df['source'] = 'facad'

CPU times: user 918 ms, sys: 15.1 ms, total: 933 ms
Wall time: 23.3 s


In [128]:
address_df = pilot_df[['address_formatted', 'lat', 'lon', 'source']].append(
    pd.DataFrame(facad_df[['address_formatted', 'lat', 'lon', 'source']]), sort=False)\
    .drop_duplicates()

In [239]:
%%time
df = pd.DataFrame(address_df['address_formatted'].dropna().str.split(', ').values.tolist(),
            columns=['street address', 'city', 'zip', 'country']).drop(columns=['country'])
df['state'] = 'TN'
df['zip'] = df['zip'].str.replace('TN ','')
df = df[['street address','city','state','zip']]
df.to_csv('geocode.csv', index=True, index_label='unique id')
gc_ab = censusgeocode.addressbatch('geocode.csv')

CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 8.17 s


In [254]:
df_index = pd.DataFrame([(i['id'],i['tract'], i['block']) for i in gc_ab], 
                  columns=['index', 'tract', 'block']).dropna()
df_index['index'] = df_index['index'].astype(int)
df_index = df_index.set_index('index')
df_index = df_index.sort_index()
df_merged = df.merge(df_index, how='left', left_index=True, right_index=True)
df_merged['address_formatted'] = df_merged['street address']+', '+df_merged['city']+', '+df_merged['state']+' '+df_merged['zip'].astype(str)+', USA'
# Merge tract & block group to original df
pilot_df = pilot_df.merge(df_merged[['address_formatted', 'tract','block']], 
                          how='left', on='address_formatted').dropna(subset=['tract'])
facad_df = facad_df.merge(df_merged[['address_formatted', 'tract','block']], 
                          how='left', on='address_formatted').dropna(subset=['tract'])

In [324]:
pilot_df['tract'].dropna().unique()

array(['000100', '006900', '001500', '006800', '003000', '006700',
       '002300', '002200', '003700', '006600'], dtype=object)

In [354]:
income_pov_keys = s3.list_objects_v2(Bucket = knox_census_bucket, Prefix='acs5/ratio_of_income_to_poverty_level_past_12_months')['Contents']
income_pov_keys = [i['Key'] for i in income_pov_keys if 'summary' in i['Key']]
df_lst = []
for i in income_pov_keys:
    year = i.split('/')[-1].split('.')[0].split('_')[-1]
    df = read_csv_s3(knox_census_bucket, i).drop(columns='percent_over_1')
    df['year'] = year
    df_lst.append(df)
pov_df = pd.concat(df_lst)
# pov_df = pov_df.pivot(index='census_tract', columns='year')
# pov_df.columns = pov_df.columns.droplevel()
# pov_df = pov_df.reset_index()
# pov_df.head()

In [430]:
pov_df.head()

,census_tract,percent_under_1,year
0,46.10,19.330454,2010
1,46.11,5.824259,2010
2,46.12,3.739907,2010
3,46.13,10.894864,2010
4,46.14,4.208348,2010


In [478]:
temp = pov_df.copy()
temp['census_tract'] = (temp['census_tract']*100).astype(int).astype(str)
temp = temp.pivot(index='year', columns='census_tract', values='percent_under_1')
temp.columns = temp.columns.get_level_values(0)
temp.columns = [''.join(col).strip() for col in temp.columns.values]
temp = temp.reset_index(drop=True)
temp = temp.dropna(axis=1)
# Get Slopes to indicate amount of poverty
slopes = temp.apply(lambda x: np.polyfit(temp.index, x, 1)[0])
slope_tract = pd.DataFrame(slopes, columns=['slope']).reset_index()
slope_tract.columns = ['tract', 'slope']

In [479]:
slope_tract

,tract,slope
0,100,-0.760946
1,1400,0.187652
2,1500,1.311218
3,1600,-0.559577
4,1700,1.849810
5,1800,0.064122
6,1900,2.909374
7,2000,1.881144
8,2100,3.413029
9,2200,0.945653
